In [31]:
# flake8: noqa: E402, B018, F401
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [32]:
import cProfile
import pstats

from icecream import ic
from memory_profiler import profile

from classes import (
    Regular,
    Slotted,
    Weak,
    attr_reader,
    attr_writer,
    bench,
    invoker,
    profile_deco,
)

N_RUNS = 10
N_LOOPS = 1_000

profiled_invoker = profile(invoker)

In [33]:
profiled_invoker(Weak)

Filename: /home/ari/workdir/deep_python_23b_aalitvinov/08/classes.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    38     77.5 MiB     77.5 MiB           1   def invoker(cls: type) -> Any:
    39     77.9 MiB      0.0 MiB           2       return cls(
    40     77.9 MiB      0.4 MiB       20005           {rng.random() for _ in range(SET_LEN)}, {rng.random() for _ in range(SET_LEN)}
    41                                             )




In [41]:
%mprun -f attr_writer attr_writer(Weak)

Filename: /home/ari/workdir/deep_python_23b_aalitvinov/08/classes.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    50     78.9 MiB     78.9 MiB           1   def attr_writer(cls: type) -> None:
    51     78.9 MiB      0.0 MiB           1       inst = invoker(cls)
    52     78.9 MiB      0.0 MiB           1       inst.first = {rng.random()}
    53     78.9 MiB      0.0 MiB           1       inst.second = {rng.random()}

In [42]:
%mprun -f attr_writer attr_writer(Slotted)

Filename: /home/ari/workdir/deep_python_23b_aalitvinov/08/classes.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    50     78.9 MiB     78.9 MiB           1   def attr_writer(cls: type) -> None:
    51     78.9 MiB      0.0 MiB           1       inst = invoker(cls)
    52     78.9 MiB      0.0 MiB           1       inst.first = {rng.random()}
    53     78.9 MiB      0.0 MiB           1       inst.second = {rng.random()}

In [14]:
%%memit
bench(lambda: invoker(Regular), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: invoker(Slotted), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: invoker(Weak), n_runs=N_RUNS, n_loops=N_LOOPS)

593 ns ± 151.9 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 462 ns
470 ns ± 31.6 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 440 ns
720 ns ± 45.1 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 663 ns
peak memory: 65.03 MiB, increment: 0.00 MiB


In [15]:
%%memit
bench(lambda: attr_reader(Regular), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: attr_reader(Slotted), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: attr_reader(Weak), n_runs=N_RUNS, n_loops=N_LOOPS)

648 ns ± 128.8 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 539 ns
647 ns ± 200.3 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 495 ns
850 ns ± 62.6 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 772 ns
peak memory: 65.05 MiB, increment: 0.00 MiB


In [16]:
%%memit
bench(lambda: attr_writer(Regular), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: attr_writer(Slotted), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: attr_writer(Weak), n_runs=N_RUNS, n_loops=N_LOOPS)

838 ns ± 41.8 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 794 ns
786 ns ± 47.3 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 735 ns
1191 ns ± 126.0 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 1061 ns
peak memory: 65.09 MiB, increment: 0.00 MiB


In [14]:
with cProfile.Profile() as pr:
    invoker(Regular)

In [15]:
pr.print_stats()

         6 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 cProfile.py:118(__exit__)
        1    0.000    0.000    0.000    0.000 classes.py:15(__init__)
        1    0.000    0.000    0.000    0.000 classes.py:34(invoker)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        2    0.000    0.000    0.000    0.000 {method 'random' of '_random.Random' objects}




In [16]:
@profile_deco
def add(a, b):
    return a + b


add(1, 2)
add(4, 5)
add.print_stat()

         4 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 3703119923.py:1(add)
        2    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


